## Combine Domain Parquet Files

This notebook aims to take the domain parquet data and converts it into a single parquet.

First we perform necessary imports.

In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import json
import re
import csv
from functools import reduce

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Crime Curated")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

PAGE_DIRECTORY = '../data/raw/domain/pages'

your 131072x1 screen size is bogus. expect trouble
24/09/09 10:42:38 WARN Utils: Your hostname, TABLET-K04EDL4Q resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/09 10:42:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/09 10:42:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# this is just used to make sure the data is correct when being read in

listings_links_path = "../data/raw/domain/metadata/listings_links.csv"

# Open and read the CSV file
with open(listings_links_path, 'r') as file:
    listing_urls = set(file.readlines())


# load all the dataframes
BATCH_SIZE = 500
dfs = []


for i in range(0, len(listing_urls), BATCH_SIZE):
    page_directory = PAGE_DIRECTORY + f'/{i}-{i+BATCH_SIZE}.parquet'
    dfs.append(spark.read.parquet(page_directory))

union_df = reduce(lambda df1, df2: df1.union(df2), dfs)

In [15]:
output_path = "../data/raw/domain/domain_current_listings.parquet"

# repartition the dataframe (as 450+ parititons is very excessive)
repartitioned = union_df.coalesce(4)

# finally write to file
repartitioned.write.parquet(output_path)

In [14]:
repartitioned.where(F.col('car_parks') > 0)

bedrooms,bathrooms,car_parks,url,name,cost_text,type,latitude,longitude
2,1,1,https://www.domai...,207/173 City Road...,$900 per week Gre...,Apartment / Unit ...,-37.8244974,144.962227
1,1,1,https://www.domai...,616 Glenferrie Ro...,$750 Per Week - F...,Apartment / Unit ...,-37.8235172,145.0356866
2,2,1,https://www.domai...,901c/851 Whitehor...,$750/w with Carpa...,Apartment / Unit ...,-37.8169818,145.1176698
2,2,1,https://www.domai...,1/378-390 St Geor...,$700 per week,Apartment / Unit ...,-37.7784848,144.9882531
2,1,2,https://www.domai...,1/16 Banksia Cres...,$390 per week,Apartment / Unit ...,-37.8610294,144.6848746
2,1,1,https://www.domai...,216/288 Adderley ...,$720 per week,Apartment / Unit ...,-37.8072029,144.9431762
1,1,1,https://www.domai...,101/288 Tooronga ...,Stay 1 mth plus -...,Apartment / Unit ...,-37.8526724,145.0424648
2,1,1,https://www.domai...,171/28 Little Lon...,$650.00,Apartment / Unit ...,-37.8085836,144.9706298
1,1,1,https://www.domai...,413A/72 Ascot Val...,$500pw/$2172pcm,Apartment / Unit ...,-37.7840953,144.9230675
1,1,1,https://www.domai...,218A/2A Duffy Str...,$495 per week,Apartment / Unit ...,-37.7401486,144.9000425


All the functions used are defined below: